In [1]:
#This Notebook explain the implementation of Decision Tree Classifier from Scratch using Information Gain(Entropy) and Gini Impurity
import numpy as np # linear algebra
import pandas as pd
from collections import Counter
from statistics import mode
import math
from sklearn.datasets import load_iris

This data and the explaintion is based on the following videa
https://www.youtube.com/watch?v=_L39rN6gz7Y

In [2]:
data=[["Yes","Yes",7,"No"],
      ["Yes","No",12,"No"],
      ["No","Yes",18,"Yes"],
      ["No","Yes",35,"Yes"],
      ["Yes","Yes",38,"Yes"],
      ["Yes","No",50,"No",],
      ["No","No",83,"No"]]
data_df=pd.DataFrame(data, columns=["Loves Popcorn", "Loves Soda", "Age", "Loves Cool as Ice"])
target="Loves Cool as Ice"

Since the method to determine the split is different between continuous and categorical values,we will split the data into nunerical and continous

In [3]:
df_cat=data_df.select_dtypes(include=np.object_).columns
df_num=data_df.select_dtypes(include=np.number).columns


In [4]:
#This class is used to create the node for each depth in the tree
class Node():
    def __init__(self, val=None,left=None,right=None,threshold=None,feature=None,gini=None):
        self.value=val
        self.left=left
        self.right=right
        self.threshold=threshold
        self.feature=feature
        self.gini=gini
        
 
 #This is the Main Tree Class which will be used to buld the tree 
      
class tree_inbuilt():
    
    def __init__(self, depth=3, leaves=3, criteria="gini"):
        self.max_depth = depth
        self.min_leaves=leaves
        self.root=None
        self.criteria=criteria
      
    
    def build_tree(self,df, depth):
        if ((depth == self.max_depth) or (len(df) <= self.min_leaves) or (tree_inbuilt.is_same(df[target].to_list()))):
            leaf=Node(mode(df[target]))
            return leaf
        else:
            if (self.criteria == "ig"):
                var,threshold, gini=tree_inbuilt.best_split_ig(df)
            else:
                var,threshold, gini=tree_inbuilt.best_split_gini(df)
            left_node_df,right_node_df=tree_inbuilt.get_node(df,var,threshold)
            left=self.build_tree(left_node_df,depth+1)
            right=self.build_tree(right_node_df,depth+1)
            final_node=Node(left=left, right=right, threshold=threshold, gini=gini, feature=var)
            return final_node
    
    def build(self, df, depth):
        self.root=self.build_tree(df,depth)
    
    def print_tree(self,Node):
        if Node is not None:
            if Node.feature is not None:
                print(f"Feature: {Node.feature}, Threshold: {Node.threshold}, criteria: {self.criteria}, Gini/IG: {Node.gini}")
            else:
                print(Node.value)
        if Node.left is not None:
            self.print_tree( Node.left)
        if Node.right is not None:
            self.print_tree(Node.right)
        
        return 
    
    def predict_tree(self,Node, df_predict):
        if Node.left is None and Node.right is None:
            return Node.value
        else:
            is_left= tree_inbuilt.determine_node_dir(df_predict,Node.feature,Node.threshold)
            if (is_left == True):
                if Node.left is not None:
                    return self.predict_tree(Node.left, df_predict)
                else:
                    return Node.value
            else:
                if Node.right is not None:
                    return self.predict_tree(Node.right,df_predict)
                else:
                    return Node.value
        return
    
    def predict(self,df_predict):
        result= self.predict_tree(self.root,df_predict)
        return result
        
    def printtree(self):
        self.print_tree(self.root)
    
    #This method is used to calculate the Gini Impurity for each branch
    @staticmethod
    def calc_gini_impurity(leaf):
        gi=0
        #get the total elements in the leaf
        total=len(leaf)
        if (total>0):
            grp=dict(Counter(leaf))
            s=0
            for k,v in grp.items():
                s= s + (v/total)**2 
            gi= 1-s
        return gi   
    
    #This method is used to calculate the Gini Impurity for each split
    @staticmethod
    def calc_weighted_gini(left_leaf, right_leaf, total):
        left_gini_imp=tree_inbuilt.calc_gini_impurity(left_leaf)
        right_gini_imp=tree_inbuilt.calc_gini_impurity(right_leaf)
        weighted_gini= ((len(left_leaf)*left_gini_imp) + (len(right_leaf)*right_gini_imp))/total
        return weighted_gini
    
    #Method to calculate the Gini Impurity for the Categorical Values
    @staticmethod
    def determine_gini_cat(var,df):
    #determine the possible values in the col. That will be the possible ways to split the column
        col=df[var]
        threshold=list(set(col))
        best_split=""
        min_gini=100000000000
        for t in threshold:
            left_leaf=df[df[var] == t][target]
            right_leaf=df[df[var]!=t][target]
            gini_gain=tree_inbuilt.calc_weighted_gini(left_leaf,right_leaf,len(df[var]))
            if (gini_gain < min_gini):
                min_gini=gini_gain
                best_split=t
        return best_split,min_gini  
    
    #Method to calculate the Gini Impurity for the Numerical Values
    @staticmethod
    def determine_gini_num(var,df):
    #determine the possible values in the col. That will be the possible ways to split the column
        col=df[var]
        col=col.sort_values()
        threshold=col.rolling(2).mean()
        threshold=threshold[1:] #drop the NaNs
        best_split=""
        min_gini=100000000000
        for t in threshold:
            left_leaf=df[df[var] <= t][target]
            right_leaf=df[df[var] > t][target]
            gini_gain=tree_inbuilt.calc_weighted_gini(left_leaf,right_leaf,len(df[var]))
            if (gini_gain < min_gini):
                min_gini=gini_gain
                best_split=t
        return best_split,min_gini   
    
    #Method to calculate Gini impurity for Categorical and Numerical Values
    @staticmethod
    def determine_gini_var(var, df):
    #determine the possible values in the col. That will be the possible ways to split the column
        if var in df_cat:
            return (tree_inbuilt.determine_gini_cat(var,df))
        else:
            return (tree_inbuilt.determine_gini_num(var,df))  
        
    
    #Determine the left and right node if the split is based on categorical variable
    @staticmethod
    def get_node_cat(df,var,threshold):
        left_node=df[df[var]== threshold]
        right_node=df[df[var] != threshold]
        return left_node,right_node
    
    #Determine the left and right node if the split is based on Numerical variable
    @staticmethod
    def get_node_num(df,var,threshold):
        left_node=df[df[var] <= threshold]
        right_node=df[df[var] > threshold]
        return left_node,right_node
    
    #Determine the Left and right Node
    @staticmethod
    def get_node(df,var,threshold):
        if var in df_cat:
            return tree_inbuilt.get_node_cat(df,var,threshold)
        else:
            return tree_inbuilt.get_node_num(df,var,threshold)
    
    #Check if all the target elements are same.
    @staticmethod
    def is_same(l):
        return all(i==l[0] for i in l)
    
    #Check whether to traverse in left or right node
    @staticmethod
    def determine_node_dir(var,threshold):
        is_left=False
        if var in df_cat:
            if (df_predict.at[0,var] == threshold):
                is_left=True
        else:
            if (df_predict.at[0,var] <=threshold):
                is_left = True
        return is_left   
    
    @staticmethod
    def determine_node_dir(df_predict,var,threshold):
        is_left=False
        if var in df_cat:
            if (df_predict.at[0,var] == threshold):
                is_left=True
        else:
            if (df_predict.at[0,var] <=threshold):
                is_left = True
        return is_left
    #Determine best split based on Gini Impurity
    @staticmethod
    def best_split_gini(df):
        features=[c for c in df.columns.to_list() if c != target]
        split_list_feature=[]
        split_list_threshold=[]
        split_list_gini=[]
        for f in features:
            split,gini=tree_inbuilt.determine_gini_var(f,df)
            split_list_feature.append(f)
            split_list_threshold.append(split)
            split_list_gini.append(gini)
        index=[i for i,v in enumerate(split_list_gini) if v== min(split_list_gini)][0]  
        return split_list_feature[index], split_list_threshold[index],split_list_gini[index]
    
    #Determine the Entropy for each node
    @staticmethod
    def calc_entropy(leaf):
        ep=0
        #get the total elements in the leaf
        total=len(leaf)
        if (total>0):
            grp=dict(Counter(leaf))
            s=0
            for k,v in grp.items():
                if (v !=0):
                    s= s + ((v/total)*math.log2(v/total))
            ep= -s
        return ep   
    
    #Calculate the Information Gain for the Split
    @staticmethod
    def calc_information_gain(parent, left_leaf,right_leaf):
        ep_parent=tree_inbuilt.calc_entropy(parent)
        ep_left= tree_inbuilt.calc_entropy(left_leaf)
        ep_right= tree_inbuilt.calc_entropy(right_leaf)
        total=len(left_leaf) + len(right_leaf)
        ep_child= ((len(left_leaf)*ep_left) + (len(right_leaf)*ep_right))/total
        ig = ep_parent - ep_child
        return ig
    
    #Determine the Information Gain for Categorical Variables
    @staticmethod
    def determine_ig_cat(var,df):
    #determine the possible values in the col. That will be the possible ways to split the column
        col=df[var]
        threshold=list(set(col))
        best_split=""
        max_ig=-0.0000001
        for t in threshold:
            left_leaf=df[df[var] == t][target]
            right_leaf=df[df[var]!=t][target]
            ig_gain=tree_inbuilt.calc_information_gain(df[target],left_leaf,right_leaf)
            if (ig_gain > max_ig):
                max_ig=ig_gain
                best_split=t
        return best_split,max_ig    

    #Determine the Information Gain for Numerical Variables
    @staticmethod
    def determine_ig_num(var,df):
        #determine the possible values in the col. That will be the possible ways to split the column
        col=df[var]
        col=col.sort_values()
        threshold=col.rolling(2).mean()
        threshold=threshold[1:] #drop the NaNs
        best_split=""
        max_ig=-0.0000001
        for t in threshold:
            left_leaf=df[df[var] <= t][target]
            right_leaf=df[df[var] > t][target]
            ig_gain=tree_inbuilt.calc_information_gain(df[target],left_leaf,right_leaf)
            if (ig_gain > max_ig):
                max_ig=ig_gain
                best_split=t
        return best_split,max_ig   

    #Determine the Information Gain
    @staticmethod    
    def determine_ig_var(var, df):
        #determine the possible values in the col. That will be the possible ways to split the column
        if var in df_cat:
            return (tree_inbuilt.determine_ig_cat(var,df))
        else:
            return (tree_inbuilt.determine_ig_num(var,df))     

    #Determine best split based on Information Gain  
    @staticmethod
    def best_split_ig(df):
        features=[c for c in df.columns.to_list() if c != target]
        split_list_feature=[]
        split_list_threshold=[]
        split_list_gini=[]
        for f in features:
            #split,gini=determine_gini_var(f,df)
            split,gini=tree_inbuilt.determine_ig_var(f,df)
            split_list_feature.append(f)
            split_list_threshold.append(split)
            split_list_gini.append(gini)

        index=[i for i,v in enumerate(split_list_gini) if v== max(split_list_gini)][0]  
        return split_list_feature[index], split_list_threshold[index],split_list_gini[index] 

In [5]:
t=tree_inbuilt(criteria="ig")
t.build(data_df,0)
t.printtree()

Feature: Loves Soda, Threshold: No, criteria: ig, Gini/IG: 0.5216406363433185
No
Feature: Age, Threshold: 12.5, criteria: ig, Gini/IG: 0.8112781244591328
No
Yes


In [6]:
df_predict=pd.DataFrame(columns=data_df.columns.to_list())
df_predict["Age"]= [15]
df_predict["Loves Soda"] = ["Yes"]
df_predict["Loves Popcorn"] = ["No"]
df_predict.head(1)
df_predict.at[0,"Loves Soda"]
df_predict["Loves Cool as Ice"]=[t.predict(df_predict)]
df_predict.head(1)

,Loves Popcorn,Loves Soda,Age,Loves Cool as Ice
0,No,Yes,15,Yes
